In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

/tmp/ipykernel_739715/2305034375.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


### my_calc_stacked_endnodes()

In [48]:
data = {
    'id': [1, 2, 3, 4],
    'linestring': [
        LineString([(0, 0), (1, 1), (2, 2)]),  # 第一条LineString坐标
        LineString([(2, 3), (3, 4), (4, 5)]),  # 第二条LineString坐标
        LineString([(5, 5), (6, 6), (7, 7)]),   # 第三条LineString坐标
        LineString([(5, 5), (6, 6), (7, 7)])
    ]
}
data = gdf = gpd.GeoDataFrame(data)
data

,id,linestring
0,1,"LINESTRING (0 0, 1 1, 2 2)"
1,2,"LINESTRING (2 3, 3 4, 4 5)"
2,3,"LINESTRING (5 5, 6 6, 7 7)"
3,4,"LINESTRING (5 5, 6 6, 7 7)"


In [49]:
matrix = [
    [-1,-1,0,1,1,3,2,4],
    [-1,-1,2,1,0,1,1,3],
    [np.nan,np.nan,-1,-1,2,1,0,2],
    [np.nan,np.nan,-1,-1,1,3,2,3],
    [np.nan,np.nan,np.nan,np.nan,-1,-1,4,1],
    [np.nan,np.nan,np.nan,np.nan,-1,-1,1,2],
    [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,-1,-1],
    [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,-1,-1]
]
matrix = pd.DataFrame(matrix)
matrix

,0,1,2,3,4,5,6,7
0,-1.0,-1.0,0.0,1.0,1.0,3.0,2,4
1,-1.0,-1.0,2.0,1.0,0.0,1.0,1,3
2,NaN,NaN,-1.0,-1.0,2.0,1.0,0,2
3,NaN,NaN,-1.0,-1.0,1.0,3.0,2,3
4,NaN,NaN,NaN,NaN,-1.0,-1.0,4,1
5,NaN,NaN,NaN,NaN,-1.0,-1.0,1,2
6,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
7,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1


In [50]:
# matrix = np.random.randint(0, 4, size=(6,6))
# matrix = matrix.astype(float)
# np.fill_diagonal(matrix, -1)
# matrix[np.tril_indices(6, -1)] = np.nan
# matrix

In [51]:
b_dist_is_zero = matrix == 0
b_dist_is_zero

,0,1,2,3,4,5,6,7
0,False,False,True,False,False,False,False,False
1,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False


In [52]:
dist_row, dist_column = np.where(b_dist_is_zero)

In [53]:
dist_row

array([0, 1, 2])

In [54]:
dist_column

array([2, 4, 6])

In [55]:
dist_combined = np.sort(np.concatenate((dist_row, dist_column)))
dist_combined

array([0, 1, 2, 2, 4, 6])

In [56]:
unique, unique_counts = np.unique(dist_combined, return_counts=True)
unique

array([0, 1, 2, 4, 6])

In [57]:
unique_counts

array([1, 1, 2, 1, 1])

In [58]:
nodes_stacked = pd.DataFrame({
        'index': unique,
        'way_ID': unique // 2, # np.ceil(unique / 2).astype(int),  # Convert indices to Wayelement ID
        'endnode1': unique % 2 == 0  # Convert indices to boolean indicator if it's endnode1
    })

nodes_stacked

,index,way_ID,endnode1
0,0,0,True
1,1,0,False
2,2,1,True
3,4,2,True
4,6,3,True


In [59]:
nodes_stacked_pairs = np.column_stack((dist_row, dist_column))
nodes_stacked_pairs

array([[0, 2],
       [1, 4],
       [2, 6]])

In [60]:
len(nodes_stacked)

5

In [61]:
# Add stacked information to dataset
i_stacked_nodes = 0
numel_way_IDs = len(nodes_stacked)

for i_ways in range(len(data)):
    if i_stacked_nodes >= numel_way_IDs:
        break
    
    if i_ways == nodes_stacked['way_ID'].iloc[i_stacked_nodes]:
        # Yes, at least one endnode is stacked
        if (i_stacked_nodes < numel_way_IDs - 1) and \
            (nodes_stacked['way_ID'].iloc[i_stacked_nodes] == nodes_stacked['way_ID'].iloc[i_stacked_nodes + 1]):
            # Both endnodes are stacked
            data.at[i_ways, 'node1_stacked'] = True
            data.at[i_ways, 'node2_stacked'] = True
            i_stacked_nodes += 1  # Skip one index
        elif nodes_stacked['endnode1'].iloc[i_stacked_nodes]:
            # Only endnode 1 is stacked
            data.at[i_ways, 'node1_stacked'] = True
            data.at[i_ways, 'node2_stacked'] = False
        else:
            # Only endnode 2 is stacked
            data.at[i_ways, 'node1_stacked'] = False
            data.at[i_ways, 'node2_stacked'] = True
        
        # Select next index to compare against way_ID
        i_stacked_nodes += 1
    else:
        # No, none of both endnodes are stacked
        data.at[i_ways, 'node1_stacked'] = False
        data.at[i_ways, 'node2_stacked'] = False

/tmp/ipykernel_739715/3954989883.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node1_stacked'] = True
/tmp/ipykernel_739715/3954989883.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node2_stacked'] = True


In [62]:
data

,id,linestring,node1_stacked,node2_stacked
0,1,"LINESTRING (0 0, 1 1, 2 2)",True,True
1,2,"LINESTRING (2 3, 3 4, 4 5)",True,False
2,3,"LINESTRING (5 5, 6 6, 7 7)",True,False
3,4,"LINESTRING (5 5, 6 6, 7 7)",True,False


### my_calc_neighbouring_endnodes()

In [37]:
data = {
    'id': [1, 2, 3, 4],
    'linestring': [
        LineString([(0, 0), (1, 1), (2, 2)]),  # 第一条LineString坐标
        LineString([(2, 3), (3, 4), (4, 5)]),  # 第二条LineString坐标
        LineString([(5, 5), (6, 6), (7, 7)]),   # 第三条LineString坐标
        LineString([(5, 5), (6, 6), (7, 7)])
    ]
}
data = gdf = gpd.GeoDataFrame(data)
data

,id,linestring
0,1,"LINESTRING (0 0, 1 1, 2 2)"
1,2,"LINESTRING (2 3, 3 4, 4 5)"
2,3,"LINESTRING (5 5, 6 6, 7 7)"
3,4,"LINESTRING (5 5, 6 6, 7 7)"


In [ ]:
matrix = [
    [-1,-1,0,1,1,3,2,4],
    [-1,-1,2,1,0,1,1,3],
    [np.nan,np.nan,-1,-1,2,1,0,2],
    [np.nan,np.nan,-1,-1,1,3,2,3],
    [np.nan,np.nan,np.nan,np.nan,-1,-1,4,2],
    [np.nan,np.nan,np.nan,np.nan,-1,-1,1,2],
    [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,-1,-1],
    [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,-1,-1]
]
matrix = pd.DataFrame(matrix)
matrix

,0,1,2,3,4,5,6,7
0,-1.0,-1.0,0.0,1.0,1.0,3.0,2,4
1,-1.0,-1.0,2.0,1.0,0.0,3.0,1,3
2,NaN,NaN,-1.0,-1.0,2.0,1.0,0,2
3,NaN,NaN,-1.0,-1.0,1.0,3.0,2,3
4,NaN,NaN,NaN,NaN,-1.0,-1.0,4,2
5,NaN,NaN,NaN,NaN,-1.0,-1.0,1,2
6,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
7,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1


In [39]:
neighbourhood_threshold = 2
b_dist_neighbourhood = (matrix < neighbourhood_threshold) & (matrix > 0)
b_dist_neighbourhood

,0,1,2,3,4,5,6,7
0,False,False,False,True,True,False,False,False
1,False,False,False,True,False,False,True,False
2,False,False,False,False,False,True,False,False
3,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,True,False
6,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False


In [40]:
nodes_neighbour_indices = np.argwhere(b_dist_neighbourhood)
nodes_neighbour_indices

array([[0, 3],
       [0, 4],
       [1, 3],
       [1, 6],
       [2, 5],
       [3, 4],
       [5, 6]])

In [41]:
dist_combined = np.concatenate((nodes_neighbour_indices[:, 0], nodes_neighbour_indices[:, 1]))
dist_combined

array([0, 0, 1, 1, 2, 3, 5, 3, 4, 3, 6, 5, 4, 6])

In [42]:
unique, unique_counts = np.unique(dist_combined, return_counts=True)
unique

array([0, 1, 2, 3, 4, 5, 6])

In [43]:
unique_counts

array([2, 2, 1, 3, 2, 2, 2])

In [44]:
nodes_neighbouring = pd.DataFrame({
        'index': unique,
        'way_ID': unique // 2,
        'endnode1': unique % 2 == 0
    })
nodes_neighbouring

,index,way_ID,endnode1
0,0,0,True
1,1,0,False
2,2,1,True
3,3,1,False
4,4,2,True
5,5,2,False
6,6,3,True


In [45]:
nodes_neighbouring_pairs = nodes_neighbour_indices.tolist()
nodes_neighbouring_pairs

[[0, 3], [0, 4], [1, 3], [1, 6], [2, 5], [3, 4], [5, 6]]

In [ ]:
data['node1_neighbour'] = False
data['node2_neighbour'] = False

# Iterate over the data to check for neighbourhood
i_neighbouring_nodes = 0
numel_way_IDs = len(nodes_neighbouring)

for i_ways in range(len(data)):
    if i_neighbouring_nodes >= numel_way_IDs:
        break
    
    if i_ways == nodes_neighbouring['way_ID'].iloc[i_neighbouring_nodes]:
        # Check if both endnodes are neighbours
        if (i_neighbouring_nodes < numel_way_IDs - 1 and 
            nodes_neighbouring['way_ID'].iloc[i_neighbouring_nodes] == 
            nodes_neighbouring['way_ID'].iloc[i_neighbouring_nodes + 1]):
            
            data.at[i_ways, 'node1_neighbour'] = True
            data.at[i_ways, 'node2_neighbour'] = True
            i_neighbouring_nodes += 1  # Skip one index
        
        elif nodes_neighbouring['endnode1'].iloc[i_neighbouring_nodes]:
            data.at[i_ways, 'node1_neighbour'] = True
            data.at[i_ways, 'node2_neighbour'] = False
        else:
            data.at[i_ways, 'node1_neighbour'] = False
            data.at[i_ways, 'node2_neighbour'] = True
        
        i_neighbouring_nodes += 1
    else:
        data.at[i_ways, 'node1_neighbour'] = False
        data.at[i_ways, 'node2_neighbour'] = False
    

In [47]:
data

,id,linestring,node1_neighbour,node2_neighbour
0,1,"LINESTRING (0 0, 1 1, 2 2)",True,True
1,2,"LINESTRING (2 3, 3 4, 4 5)",True,True
2,3,"LINESTRING (5 5, 6 6, 7 7)",True,True
3,4,"LINESTRING (5 5, 6 6, 7 7)",True,False


### my_group_nodes()

In [109]:
nodes_stacked_pairs

array([[0, 2],
       [1, 4],
       [2, 6]])

In [110]:
nodes_stacked_pairs1 = [[3, 2],
       [1, 4],
       [2, 6]]
nodes_stacked_pairs1

[[3, 2], [1, 4], [2, 6]]

In [111]:
# 对每个节点对进行排序。比如 (1, 2) 和 (2, 1) 在排序后都变成 [1, 2]
pairs_sorted = [sorted(pair) for pair in nodes_stacked_pairs]
pairs_sorted

[[0, 2], [1, 4], [2, 6]]

In [112]:
pairs_sorted.sort()
pairs_sorted

[[0, 2], [1, 4], [2, 6]]

In [113]:
list_groups = []

In [114]:
for partner1, partner2 in pairs_sorted:
    print(partner1, partner2)
    list_groups.append(sorted([partner1, partner2]))

0 2
1 4
2 6


In [115]:
list_groups

[[0, 2], [1, 4], [2, 6]]

In [116]:
for partner1, partner2 in pairs_sorted:
    row_partner1 = row_partner2 = None

    # Check if partner1 and partner2 already belong to any groups
    for i, group in enumerate(list_groups):
        if partner1 in group:
            row_partner1 = i
        if partner2 in group:
            row_partner2 = i

    if row_partner1 is not None:
        if row_partner2 is not None:
            if row_partner1 == row_partner2:
                continue  # Both are in the same group, no action needed
            else:
                # Merge groups and remove the redundant group
                list_groups[row_partner1] = sorted(set(list_groups[row_partner1] + list_groups[row_partner2]))
                del list_groups[row_partner2]
        else:
            # Add partner2 to the group containing partner1
            list_groups[row_partner1].append(partner2)
            list_groups[row_partner1] = sorted(set(list_groups[row_partner1]))
    elif row_partner2 is not None:
        # Add partner1 to the group containing partner2
        list_groups[row_partner2].append(partner1)
        list_groups[row_partner2] = sorted(set(list_groups[row_partner2]))
    else:
        # Create a new group for both partners
        list_groups.append(sorted([partner1, partner2]))

In [118]:
list_groups

[[0, 2, 6], [1, 4]]

### group_stacked_endnodes()

In [119]:
data['ID_node1_grouped'] = np.nan
data['ID_node2_grouped'] = np.nan
data['lon1_grouped'] = np.nan
data['lat1_grouped'] = np.nan
data['lon2_grouped'] = np.nan
data['lat2_grouped'] = np.nan
data

,id,linestring,node1_stacked,node2_stacked,ID_node1_grouped,ID_node2_grouped,lon1_grouped,lat1_grouped,lon2_grouped,lat2_grouped
0,1,"LINESTRING (0 0, 1 1, 2 2)",True,True,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"LINESTRING (2 3, 3 4, 4 5)",True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"LINESTRING (5 5, 6 6, 7 7)",True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"LINESTRING (5 5, 6 6, 7 7)",True,False,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
list_groups

[[0, 2, 6], [1, 4]]

In [121]:
# 遍历 stacked group
for group in list_groups:
    # 获取当前 group 的首个成员 node ID
    first_node_id = group[0]
    way_id = first_node_id // 2  # 在 Python 中，节点 ID 从 0 开始
    is_node1 = (first_node_id % 2) == 0  # 偶数 ID 判定为起点

    # 获取起点或终点的坐标信息
    if is_node1:
        # 起点
        grouped_node_id = data.at[way_id, 'nodes'][0]  # 起点
        grouped_lon, grouped_lat = data.at[way_id, 'geometry'].coords[0]
    else:
        # 终点
        grouped_node_id = data.at[way_id, 'nodes'][-1]  # 终点
        grouped_lon, grouped_lat = data.at[way_id, 'geometry'].coords[-1]

    # 更新 group 中的所有成员
    for member_node_id in group:
        way_id = member_node_id // 2  # 计算对应的 way ID
        is_node1 = (member_node_id % 2) == 0  # 偶数 ID 判定为起点

        if is_node1:
            # 更新起点
            data.at[way_id, 'ID_node1_grouped'] = grouped_node_id
            data.at[way_id, 'lon1_grouped'] = grouped_lon
            data.at[way_id, 'lat1_grouped'] = grouped_lat
        else:
            # 更新终点
            data.at[way_id, 'ID_node2_grouped'] = grouped_node_id
            data.at[way_id, 'lon2_grouped'] = grouped_lon
            data.at[way_id, 'lat2_grouped'] = grouped_lat

KeyError: 'nodes'